# Keycloak OIDC Experiments

> Testing Keycloak OIDC flows

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import base64
import hashlib
import html
import json
import os
import re
import urllib.parse
import requests

In [ ]:
provider = "https://localhost:8037/auth/realms/test"
client_id = "test"
redirect_uri = "https://localhost:3737"

In [ ]:
code_verifier = base64.urlsafe_b64encode(os.urandom(40)).decode('utf-8')
code_verifier = re.sub('[^a-zA-Z0-9]+', '', code_verifier)
code_verifier, len(code_verifier)

('uM3yNkRpclzUuABiVl64aNe6NAgqPyiah64BWEyNJwute7YcbFg', 51)

In [ ]:
code_challenge = hashlib.sha256(code_verifier.encode('utf-8')).digest()
code_challenge = base64.urlsafe_b64encode(code_challenge).decode('utf-8')
code_challenge = code_challenge.replace('=', '')
code_challenge, len(code_challenge)

('yvB91DCUFnWp3VcJUai8nD_9vX6xmZ2_fJianKpnh7g', 43)

In [ ]:
state = "fooobarbaz"
resp = requests.get(
    url=provider + "/protocol/openid-connect/token",
    params={
        "response_type": "code",
        "client_id": client_id,
        "scope": "openid",
        "redirect_uri": redirect_uri,
        "state": state,
        "code_challenge": code_challenge,
        "code_challenge_method": "S256",
    },
    verify=False,
    allow_redirects=False
)
resp.status_code

/Users/admin/Library/Caches/pypoetry/virtualenvs/explore-keycloak-hV1l5bXr-py3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


404

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()